# フェーズ1 - 3週目：n量子ビットへの拡張と測定機能の実装 ⚙️

シミュレータ開発の核心部分です。これまでのロジックを一般化し、任意のビット数 **n** に対応させます。さらに、量子計算の結果を取り出すための**測定**を実装します。

今回からコードの再利用性と管理のしやすさを高めるために、**クラス**としてシミュレータを設計します。

## 1\. ライブラリのインポート

今回は測定結果の集計に便利な `collections.Counter` もインポートします。

In [1]:
import numpy as np
from collections import Counter

## 2\. n量子ビットシミュレータのクラス設計

`QuantumSimulator` というクラスを作成し、量子ビット数や状態ベクトルをインスタンス変数として保持するようにします。

### n量子ビット系でのゲート操作

n量子ビットの系で、特定の **k番目** の量子ビットにゲート `U` を適用する場合、システム全体に作用する演算子（行列）は、以下のようなテンソル積で構築されます。

$$\text{Operator} = I^{\otimes k} \otimes U \otimes I^{\otimes (n-k-1)}$$

例えば、3量子ビット系で1番目（0-indexed）の量子ビットにXゲートを適用する場合の演算子は $I \otimes X \otimes I$ となります。これを一般化するロジックをクラス内に実装します。

In [2]:
class QuantumSimulator:
    """
    n量子ビットの状態をシミュレーションするクラス
    """
    def __init__(self, n_qubits):
        if n_qubits <= 0:
            raise ValueError("量子ビット数は1以上でなければなりません。")
        self.n_qubits = n_qubits
        # 状態ベクトルを |0...0> で初期化 (サイズは 2^n)
        self.state_vector = np.zeros(2**self.n_qubits, dtype=complex)
        self.state_vector[0] = 1 + 0j

    def _get_operator(self, gate, target_qubit):
        """特定の量子ビットに作用するゲートの、システム全体の演算子を生成する"""
        gate_i = np.identity(2, dtype=complex)
        
        op_list = [gate_i] * self.n_qubits
        op_list[target_qubit] = gate
        
        operator = op_list[0]
        for i in range(1, self.n_qubits):
            operator = np.kron(operator, op_list[i])
            
        return operator

    def apply_gate(self, gate, target_qubit):
        """指定した量子ビットに1量子ビットゲートを適用する"""
        if target_qubit < 0 or target_qubit >= self.n_qubits:
            raise ValueError("ターゲット量子ビットのインデックスが範囲外です。")
        operator = self._get_operator(gate, target_qubit)
        self.state_vector = np.dot(operator, self.state_vector)

## 3\. 測定機能の実装 🎲

量子計算の最終ステップは測定です。量子状態は決定論的ではなく、**確率的**に振る舞います。

状態ベクトルが $|\psi\rangle = \sum_{i=0}^{2^n-1} c_i |i\rangle$ で与えられるとき、基底状態 $|i\rangle$ が測定される確率は、$P(i) = |c_i|^2$ となります。

測定機能は、この確率分布に従ってランダムサンプリングを行うことでシミュレートします。

In [3]:
# QuantumSimulatorクラスにmeasureメソッドを追加
def measure(self, shots=1):
    """現在の状態ベクトルに基づいて測定を行う"""
    probabilities = np.abs(self.state_vector)**2
    outcomes = np.arange(2**self.n_qubits)
    results_decimal = np.random.choice(outcomes, size=shots, p=probabilities)
    results_binary = [format(res, f'0{self.n_qubits}b') for res in results_decimal]
    return results_binary

# 作成したメソッドをクラスに追加
QuantumSimulator.measure = measure

## 4\. シミュレーション実行！

それでは、3量子ビットのシミュレータを動かしてみましょう。
以下の回路を実行し、測定結果がどうなるか確認します。

1.  初期状態 $|000\rangle$
2.  0番目の量子ビットに **H (アダマール) ゲート**を適用
3.  2番目の量子ビットに **X (NOT) ゲート**を適用

この操作により、状態は $\frac{1}{\sqrt{2}}(|001\rangle + |101\rangle)$ となるはずです。
つまり、測定結果は `'001'` と `'101'` がおよそ50%ずつの確率で現れると予測されます。

In [4]:
# --- シミュレーションの準備 ---
N_QUBITS = 3
sim = QuantumSimulator(N_QUBITS)

# 1量子ビットゲートの定義
gate_h = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
gate_x = np.array([[0, 1], [1, 0]])

# --- 回路の実行 ---
print(f"初期状態: |{ '0' * N_QUBITS }>")
sim.apply_gate(gate_h, 0) # H on q0
sim.apply_gate(gate_x, 2) # X on q2
print("H(q0), X(q2) を適用しました。")

# --- 測定の実行 ---
N_SHOTS = 1000
measurements = sim.measure(shots=N_SHOTS)
counts = Counter(measurements)

# --- 結果の表示 ---
print(f"\n--- {N_SHOTS}回の測定結果 ---")
for result, count in sorted(counts.items()):
    print(f"  '{result}': {count}回 (確率は約 {count/N_SHOTS:.2f})")

初期状態: |000>
H(q0), X(q2) を適用しました。

--- 1000回の測定結果 ---
  '001': 496回 (確率は約 0.50)
  '101': 504回 (確率は約 0.50)


予測通り、`'001'` と `'101'` だけが観測され、それぞれ約50%の確率で出現することが確認できました。

これで、シミュレータのコアとなるエンジンが完成しました。次のステップ（4週目）では、このコードをより使いやすくするために**モジュール化**（整理整頓）を行います。